### Neural Nets

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv(r"C:\Users\varun george\OneDrive\Desktop\SNU\4th sem\mlt\datasets\Bank_Personal_Loan_Modelling.csv")

In [3]:
df = df.drop(columns = ["ID","ZIP Code"])
x = df.drop("Personal Loan",axis=1)
y = df["Personal Loan"]

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=0)

x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [11]:
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense

from keras.optimizers import Adam

In [12]:
model = Sequential()
model.add(Dense(units=16, activation='relu', input_dim=x_train.shape[1]))
model.add(Dense(units=1, activation='sigmoid'))

In [13]:
def fitness(weights):
    weights = weights.flatten()
    weight = weights[:176].reshape((11, 16))
    bias= np.array([0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.])
    model.layers[0].set_weights([weight, bias])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.003),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy()])
    Loss = model.evaluate(x_train, y_train, verbose=0)
    return Loss

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 16)                192       
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 209
Trainable params: 209
Non-trainable params: 0
_________________________________________________________________


In [31]:
def ant_colony_optimization(pop_size=10, itr=20,ev_rate=0.5, update=100):
    
    # initialize the pheromone matrix
    pheromone = np.ones((12, 16))
    
    
    # initialize the best weights and the loss
    best_weights = None
    best_loss = 99
    
    for i in range(itr):
        ants = []
        ant_loss = []
        
        for j in range(pop_size):
            weights = np.zeros((12,16))
            for k in range(12):
                for l in range(16):
                    if np.random.rand() < pheromone[k][l]:
                        weights[k][l] = np.random.randn()
                        
            losses = fitness(weights)
            losses = losses[0]
            ants.append(weights)
            ant_loss.append(losses)
            
            
            if losses< best_loss:
                best_weights = weights
                best_loss = losses
                
                
        # update the pheromone matrix
        for k in range(11):
            for l in range(16):
                pheromone[k][l] = (1 - ev_rate) * pheromone[k][l]
                for j in range(pop_size):
                    if ants[j][k][l] != 0:
                        pheromone[k][l] += update / ant_loss[j] * ants[j][k][l]
       
        if i % 10 == 0:
            print("Iteration:", i, "Best Loss:", best_loss)
    
    return best_weights

In [32]:
best_weights = ant_colony_optimization()

Iteration: 0 Best Loss: 0.6661019325256348
Iteration: 10 Best Loss: 0.4766400456428528


In [33]:
best_weights = best_weights.flatten()
weight = best_weights[:176].reshape((11, 16))
bias= np.array([0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.])

model.layers[0].set_weights([weight, bias])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.003),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy()])

In [34]:
import numpy as np
y_pred = model.predict(x_test)
y_pred = np.where(y_pred>=0.5,1,0)
from sklearn.metrics import accuracy_score
accuracy_score(y_pred,y_test)

47/47 [==============================] - 0s 1ms/step


0.7746666666666666